In [3]:
import numpy as np
from pathlib import *
import re

In [38]:
reports_folder = Path("./reports/")

# must format the output folder
for i in reports_folder.iterdir():
    # remove unnecessary data from file if possible
    lines = []
    with i.open('r') as file:
        lines = file.readlines()
    slice_index1 = -1
    slice_index2 = -1
    pattern_name_value_block = re.compile(r".*(NAME).*(VALUE).*")
    pattern_facility_block = re.compile(r".*(FACILITY).*")
    pattern_header = re.compile(r".*(GPSS World Simulation Report).*")

    for index, line in enumerate(lines):
        if re.match(pattern_name_value_block, line.strip()) is not None:
            print("matched name value block ", line)
            slice_index1 = index
            continue
        if re.match(pattern_facility_block, line.strip()) is not None:
            print("matched facility block", line)
            slice_index2 = index

    if slice_index1 > -1:
        lines = lines[:slice_index1] + lines[slice_index2:]  
    indices_to_filter = list()
    for index, line in enumerate(lines):
        if re.match(pattern_header, line.strip()) is not None:
            indices_to_filter.extend(list(range(index, index + 4)))
    lines = list(filter(lambda x: x[0] not in indices_to_filter, enumerate(lines)))
    lines = list(map(lambda x: x[1], lines))

    if re.fullmatch(r".*(RMULT).*", lines[0].strip()) is None:
        print("no RMULT found")
        order_index = i.name
        gpss_generated=Path("./gpss_experiment_scripts/")
        related_file_with_mult = gpss_generated.joinpath(f"{order_index}")
        first_line = None
        with related_file_with_mult.open('r') as file:
            lines1 = file.readlines()
            first_line = lines1[0]
        lines.insert(0, first_line)

    # on the end of the file there are bad symbols
    pattern_queue_entity =re.compile(r"[\s]*[A-Z0-9_]+[\s]+[0-9]+[\s]+[0-9]+.*")

    queue_last = len(lines)
    for index, line in enumerate(lines[::-1]):
        if re.fullmatch(pattern_queue_entity, line.strip()) is not None:
            print(line)
            queue_last = -index
            break
    if (queue_last < 0):
        lines = lines[:queue_last]
    
    with i.open('w') as file:
        file.writelines(lines)

matched facility block FACILITY         ENTRIES  UTIL.   AVE. TIME AVAIL. OWNER PEND INTER RETRY DELAY

 FMS_OUT             6    0     75      0     1.013    110.337    110.337   0



In [12]:
test = """              GPSS World Simulation Report - dz.39.1
"""
test1 = """ sdafsdf GPSS sdfsdfsf """
pattern = re.compile(r".*(GPSS World Simulation Report).*")
print(re.match(pattern, test))

<re.Match object; span=(0, 52), match='              GPSS World Simulation Report - dz.3>


In [25]:
pattern =re.compile(r".*[A-Z0-9][\s]+[0-9]+[\s]+[0-9]+.*")
test = """ FMS_OUT             6    0     75      0     1.013    110.337    110.337   0
"""
re.match(pattern, test) is not None

True

In [27]:
lines[::-1]

['                   15    DEPART              75             0',
 '\x00        75             0       0\n',
 ' FMS_OUT             6    0     75      0     1.013    110.337    110.337   0\n',
 ' STANOK3_OUT         1    0     75     75     0.000      0.000      0.000   0\n',
 ' STANOK3_IN          1    0     75     75     0.000      0.000      0.000   0\n',
 ' STANOK2_OUT         1    0     75     75     0.000      0.000      0.000   0\n',
 ' STANOK2_IN          1    0     75     75     0.000      0.000      0.000   0\n',
 ' STANOK1_OUT         1    0     75     75     0.000      0.000      0.000   0\n',
 ' STANOK1_IN          1    0     75     75     0.000      0.000      0.000   0\n',
 ' FMS_IN              5    0     75      0     0.927    101.027    101.027   0\n',
 'QUEUE              MAX CONT. ENTRY ENTRY(0) AVE.CONT. AVE.TIME   AVE.(-0) RETRY\n',
 '\n',
 '\n',
 ' STANOK3             75    0.153      16.676  1        0    0    0     0      0\n',
 ' STANOK2             75    0.15